<a href="https://colab.research.google.com/github/JustAlex5/deep_learning/blob/main/ex10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning: Ex.10 - RNN

Submitted by: [... **your name and ID** ...]


In [1]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import SimpleRNN, LSTM # <--- recurrent layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap

print(tf.__version__)

2.7.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ls '/content/drive/MyDrive/Ex10'

war_and_peace.txt


In [4]:
path='/content/drive/MyDrive/Ex10/war_and_peace.txt'

***
### 1. Preprocess the text corpus

(if you are using google colab, remember to upload the corpus file first..)

In [5]:
f = open(path,'r') # open the corpus file

text = f.read().lower()  # read file and convert to lower-case letters
text= text.replace('\n', ' ')

print('len(text) = ',len(text))


print(text[:400]) # print the first 400 characters..

len(text) =  3196213
﻿"well, prince, so genoa and lucca are now just family estates of the buonapartes. but i warn you, if you don't tell me that this means war, if you still try to defend the infamies and horrors perpetrated by that antichrist--i really believe he is antichrist--i will have nothing more to do with you and you are no longer my friend, no longer my 'faithful slave,' as you call yourself! but how do you


- generate training sequences of `T=20` characters, by sampling the text corpus with a stride of 5 characters (i.e., each sequences starts 5 chars after the begining of the last sequences).

- generate a matching list, holding the `next_char` for each of your sequences.

- how many sequences did you extract in total? `N = ?`

- convert the sequences into a 1-hot representation, suitable for our model trainig:

`X.shape = (N, T, len(chars))`

`Y.shape = (N, len(chars))`

In [6]:
chars = sorted(list(set(text)))
text_size, vocab_size = len(text), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (text_size, vocab_size))
print('chars = ',chars)

There are 3196213 total characters and 56 unique characters in your data.
chars =  [' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'ä', 'é', 'ê', '\ufeff']


In [7]:
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

ix_to_char

{0: ' ',
 1: '!',
 2: '"',
 3: "'",
 4: '(',
 5: ')',
 6: '*',
 7: ',',
 8: '-',
 9: '.',
 10: '/',
 11: '0',
 12: '1',
 13: '2',
 14: '3',
 15: '4',
 16: '5',
 17: '6',
 18: '7',
 19: '8',
 20: '9',
 21: ':',
 22: ';',
 23: '=',
 24: '?',
 25: 'a',
 26: 'b',
 27: 'c',
 28: 'd',
 29: 'e',
 30: 'f',
 31: 'g',
 32: 'h',
 33: 'i',
 34: 'j',
 35: 'k',
 36: 'l',
 37: 'm',
 38: 'n',
 39: 'o',
 40: 'p',
 41: 'q',
 42: 'r',
 43: 's',
 44: 't',
 45: 'u',
 46: 'v',
 47: 'w',
 48: 'x',
 49: 'y',
 50: 'z',
 51: 'à',
 52: 'ä',
 53: 'é',
 54: 'ê',
 55: '\ufeff'}

In [8]:
    ###########################
    ###  your code here...  ###
    ###########################

T = 20  # extract training sequences of length T
stride = 5

sequences = []  # This holds our extracted sequences
next_chars = []  # This holds the targets (the follow-up character)

for i in range(0, len(text) - T, stride):
  sequences.append(text[i: i + T])
  next_chars.append(text[i + T])


In [9]:
len(sequences)

639239

In [10]:
for i in range(10):
    print(sequences[i], '->', next_chars[i])

﻿"well, prince, so g -> e
l, prince, so genoa  -> a
ince, so genoa and l -> u
 so genoa and lucca  -> a
enoa and lucca are n -> o
and lucca are now ju -> s
ucca are now just fa -> m
are now just family  -> e
ow just family estat -> e
st family estates of ->  


In [ ]:
X = np.zeros((len(sequences), T, len(text)), dtype='bool')
Y = np.zeros((len(sequences), len(text)), dtype='bool')

In [ ]:


for i, seq in enumerate(sequences):
    for t, char in enumerate(seq):
        X[i, t, char_to_ix[char]] = 1
    Y[i, char_to_ix[next_chars[i]]] = 1
    
print('X.shape = (#examples, T, input-dim) =', X.shape)
print('Y.shape = (#examples, output-dim) =', Y.shape)

***
### 2. LTSM Model

- Build an `LTSM` model with 128 (hidden)-units that accepts the input sequences. Add a `Dense` layer on top of it, with `len(chars)` softmax units.

- Train the model for only 1 epoch (use: `RMSprop` and batch size of 128).



In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

### 3. Model predictions


- use the senternce `the meaning of life ` as an input to the model (convert it to 1-hot first..),

- plot the model's output as a probability distribution over the list of chars.

- sample a single char from that distribution, and add it to the generated sentence.

- update the 1-hot buffer, and continue the process for 99 more letters (using a loop).

- print the resulting sentence.

In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
### 3. Fit your model

- Fit your model a bit more (try 10-20 epochs), and regenerate a new `N=100` sentence sample. Does it get any better?

- If you wish, you can try to train the model further, or you can try using a different corpus (dataset) for the training (you can even try a text in hebrew). Be creative ;)

In [ ]:
    ###########################
    ###  your code here...  ###
    ###########################

***
## Good Luck!